In [ ]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv()
open_api_key = os.getenv("OPENAI_API_KEY")

if open_api_key:
    print(f"Open API Key exists! It starts with {open_api_key[0:8]}")
else:
    print("Open API Key not found!")


In [ ]:
openai = OpenAI()

system_prompt = "You are a helpful assistant"

def msg_gpt(prompt):
    message = [
        {"role": "system", "content":system_prompt },
        {"role":"user", "content":prompt}
    ]
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = message,
    )
    return response.choices[0].message.content

In [ ]:
msg_gpt("Tell me a witty joke")

In [ ]:
system_prompt = "You are a helpful assistant that responds in markdown"
gr.Interface(fn=msg_gpt, 
             inputs=[gr.Textbox(label="Your message",lines=6)],
             outputs=[gr.Markdown(label="Response")],
             flagging_mode="never").launch(share=True)

In [ ]:
def stream_gpt(prompt):

    message = [
        {"role": "system", "content":system_prompt},
        {"role": "user", "content": prompt}
    ]

    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=message,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your msg", lines=5)],
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never").launch()


In [24]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [26]:
system_prompt = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [27]:
def stream_brochure(company_name, url):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    result = stream_gpt(prompt)
    yield from result

In [31]:
gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name", lines=1),
        gr.Textbox(label="Enter the url of the company: ", lines=1)
    ],
    outputs=[gr.Markdown(label="Brochure")],
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
